# Import things

In [ ]:
import os
import openai

from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

# Define the state

In [ ]:
class DebugState(TypedDict):
    question: str
    answer: str

# Create node function

In [ ]:
def qa_node(state: DebugState) -> DebugState:
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    response = llm.invoke([
        {"role": "user", "content": state["question"]}
    ])
    return {
        "question": state["question"],
        "answer": response.content
    }

# Build the graph

In [ ]:
builder = StateGraph(DebugState)
builder.add_node("qa_node", qa_node)
builder.add_edge(START, "qa_node")
builder.add_edge("qa_node", END)

# We'll set a "static breakpoint" after the qa_node executes, for debugging.
memory = MemorySaver()
graph = builder.compile(
    checkpointer=memory,
    interrupt_after=["qa_node"]
)

# Define debug function

In [ ]:
def run_debug():
    initial_state = {"question": "what is the shortest shipping route between rotterdam and osaka", "answer": ""}
    config = {"configurable": {"thread_id": "debug_thread"}}
    # 1) The graph runs 'qa_node' and then hits the breakpoint interrupt_after=qa_node
    #    so it will pause before going to END
    partial_state = graph.invoke(initial_state, config=config)
    print("Graph paused after 'qa_node'. Let's see the partial state:")
    print(partial_state)

    # Now we might manually fix or update the answer
    partial_state["answer"] += " [Debug note: Double-check citations]"
    # 2) Resume from the breakpoint to finalize the graph
    final_state = graph.invoke(partial_state, config=config)
    print("Final State:", final_state)

In [ ]:
run_debug()